In [17]:

# WARNING - disk size of data file is about 5.4GB, and after reading it into RAM
# R process uses about 6.4GB of memory.
DATA_FNAME = "clash_royale_games_Jan_2019.csv"
data = read.csv(DATA_FNAME)
DATA_LEN = length(data$timestamp)



In [26]:
transaction_summary = function(data) {
    res = aggregate(cbind(data$has_won, data$tag) ~ data$timestamp, data, id)
    bad_indices = res$V1 %in% c("0", "1", "0.5") # || res$V1 == "1"]
    library(dplyr)
    filter(res, !bad_indices)
}
data_summary = transaction_summary(data[1:(DATA_LEN),])

data_summary

In [ ]:
1

Error: Session cannot generate requests